# Вариант 1

In [135]:
import pandas as pd
import re
import random
import math
import nltk
from nltk.util import ngrams
from nltk import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [109]:
df = pd.read_csv('психология.csv')
df.head()

,url,author,likes,text,comments
0,https://whoisdutytoday.livejournal.com/20353.h...,https://whoisdutytoday.livejournal.com/,5,Психическое здоровье создателя контента: что г...,"['Так и есть.', 'А могли бы стать токарями, фр..."
1,https://lika-bo.livejournal.com/68166.html?media,https://lika-bo.livejournal.com/,24,"Примеры самообмана, мешающего нам мыслить логи...","['едва ли какой вывод, вот тебе на', 'Согласен..."
2,https://lika-bo.livejournal.com/68864.html?media,https://lika-bo.livejournal.com/,49,"Привычки, которыми мы злим окружающих.\nКому х...","['что за эта проблема, табак', 'Есть ещё привы..."
3,https://andeadd.livejournal.com/1818144.html?m...,NaN,38,.\n,[]
4,https://lashevchenko.livejournal.com/493028.ht...,https://lashevchenko.livejournal.com/,51,Три способа быстро привести себя в норму после...,['Здравствуйте! Система категоризации Живого Ж...


In [110]:
def process_text(text):
    text = text.lower()
    sentences = nltk.sent_tokenize(text)
    processed_sentences = []
    for sentence in sentences:
        cleaned_sentence = re.sub(r'[^а-яa-z\s]', '', sentence)
        tagged_sentence = '<SOS> ' + cleaned_sentence.strip() + ' <EOS>'
        processed_sentences.append(tagged_sentence)
    return processed_sentences


def process_comment(comment):
    comment = comment.split("\',")
    comment = list(map(lambda x: re.sub(r'[^а-яa-z\s]', '', x), comment))
    comment = list(map(lambda x: '<SOS> ' + x + ' <EOS>', comment))
    return comment


df['text'] = df['text'].apply(process_text)
df['comments'] = df['comments'].apply(process_comment)

In [111]:
df.head()

,url,author,likes,text,comments
0,https://whoisdutytoday.livejournal.com/20353.h...,https://whoisdutytoday.livejournal.com/,5,[<SOS> психическое здоровье создателя контента...,"[<SOS> ак и есть <EOS>, <SOS> могли бы стать..."
1,https://lika-bo.livejournal.com/68166.html?media,https://lika-bo.livejournal.com/,24,[<SOS> примеры самообмана мешающего нам мыслит...,"[<SOS> едва ли какой вывод вот тебе на <EOS>, ..."
2,https://lika-bo.livejournal.com/68864.html?media,https://lika-bo.livejournal.com/,49,[<SOS> привычки которыми мы злим окружающих <E...,"[<SOS> что за эта проблема табак <EOS>, <SOS> ..."
3,https://andeadd.livejournal.com/1818144.html?m...,NaN,38,[<SOS> <EOS>],[<SOS> <EOS>]
4,https://lashevchenko.livejournal.com/493028.ht...,https://lashevchenko.livejournal.com/,51,[<SOS> три способа быстро привести себя в норм...,[<SOS> дравствуйте истема категоризации ивого ...


In [ ]:
all_sentences = []

df.apply(lambda row: all_sentences.extend(row['text'] + row['comments']), axis=1)

In [113]:
len(all_sentences)

19366

In [114]:
train_sentences, test_sentences = train_test_split(all_sentences, test_size=0.25, random_state=42)
len(train_sentences), len(test_sentences)

(14524, 4842)

In [118]:
test_sentences[:5]

['<SOS> психологической разрядки не наступает <EOS>',
 '<SOS> другие новости так же были не слишком радужными <EOS>',
 '<SOS> то есть когда этой самой микрорнк достаточно и цепочка нейронов работает как часы и сигналы по ней проходят как положено <EOS>',
 '<SOS> нет <EOS>',
 '<SOS> беспокойный сон и ночные кошмары наблюдаются у чувствительных детей которые бурно реагируют на любой внешний раздражитель <EOS>']

In [124]:
unigrams = []
bigrams = []
for sent in test_sentences:
    tokens = sent.split()
    unigrams.extend(ngrams(tokens, n=1))
    bigrams.extend(ngrams(tokens, n=2))

In [125]:
bigrams[:5]

[('<SOS>', 'психологической'),
 ('психологической', 'разрядки'),
 ('разрядки', 'не'),
 ('не', 'наступает'),
 ('наступает', '<EOS>')]

In [126]:
bigrams_set = set(bigrams)
bigrams_distribution = {bigram : bigrams.count(bigram) / unigrams.count((bigram[0], )) for bigram in bigrams_set}
bigrams_distribution

{('подбертамо', 'возразить'): 1.0,
 ('красивые', 'засранцы'): 0.14285714285714285,
 ('короне', '<EOS>'): 0.3333333333333333,
 ('данные', 'являются'): 0.2,
 ('места', 'в'): 0.1111111111111111,
 ('атмосферном', 'давлении'): 1.0,
 ('сами', 'знаете'): 0.047619047619047616,
 ('подтвержднных', 'вредностей'): 1.0,
 ('х', 'да'): 0.125,
 ('станут', 'богатыми'): 0.125,
 ('часто', 'так'): 0.014285714285714285,
 ('заработанные', 'деньги'): 1.0,
 ('поисках', 'букета'): 0.3333333333333333,
 ('ход', 'идут'): 0.5,
 ('коронавирусом', '<EOS>'): 0.3333333333333333,
 ('файзер', 'сами'): 1.0,
 ('о', 'младенце'): 0.002570694087403599,
 ('кремом', 'в'): 1.0,
 ('тебе', 'интереснее'): 0.021739130434782608,
 ('начинают', 'привлекать'): 0.1111111111111111,
 ('радостями', 'век'): 1.0,
 ('уху', 'же'): 1.0,
 ('запахов', 'и'): 1.0,
 ('детские', 'тревожные'): 0.125,
 ('муж', 'жена'): 0.05555555555555555,
 ('склерозом', 'высказали'): 1.0,
 ('мы', 'пойдем'): 0.0048543689320388345,
 ('мужик', '<EOS>'): 0.333333333333333

In [137]:
def generate_text(start_bigram, N):
    cur = start_bigram
    generated_text = [cur[0], cur[1]]
    l = 0

    while cur[1] != '<EOS>' and l <= N:
        next_bs = [(bigram, prob) for bigram, prob in bigrams_distribution.items() if bigram[0] == cur[1]]
        if not next_bs:
            break

        next_big, prob = random.choices(next_bs, weights=[prob for _, prob in next_bs], k=1)[0]
        generated_text.append(next_big[1])
        l += len(next_big[1]) + 1
        cur = next_big

    return ' '.join(generated_text)

def calculate_perp(text):
    words = text.split()
    N = len(words)
    perp_sum = 0

    for i in range(1, N):
        bigram = (words[i-1], words[i])
        if bigram in bigrams_distribution:
            prob = bigrams_distribution[bigram]
            perp_sum -= math.log2(prob)

    perp = 2 ** (perp_sum / N)
    return perp

total = 0
c = 0

for bigram in bigrams_distribution.keys():
    if bigram[0] == '<SOS>':
        generated_text = generate_text(bigram, 100)
        perplexity = calculate_perp(generated_text)
        print(f"Текст {c}: {generated_text}")
        print(f"Перплексия: {perplexity}\n")
        total += perplexity
        c += 1

Текст 0: <SOS> likabo ебилкаозяйчики е решительность и впендюрить о необходимости получить и удобно все <EOS>
Перплексия: 24.715801399651923

Текст 1: <SOS> удивительное дело через реализацию своих пациентов с предрасположенностью к оскве осковский психологопедагогический
Перплексия: 14.57626854023919

Текст 2: <SOS> крот в коем случае вы можете доверять по дизельной электростанции и неприятных переживаний и он хоть и цепочка
Перплексия: 26.439192523354436

Текст 3: <SOS> подзаголовок донесения с дебильностью коллега вс сводится к нашим очень важно сколько ни людей просто не могут начаться
Перплексия: 19.608137054453014

Текст 4: <SOS> обратите внимание и тп в украине <EOS>
Перплексия: 22.492480711889748

Текст 5: <SOS> налаживание пищевого поведения которой отправляю всех обосрать <EOS>
Перплексия: 8.11777213300841

Текст 6: <SOS> вместо просмотра указанного произведения искусства знаменитые и зарабатывать <EOS>
Перплексия: 11.382717263456504

Текст 7: <SOS> споминается забавный факт 

KeyboardInterrupt: ignored

In [ ]:
average_perplexity = total / c
print(f"Средняя перплексия: {average_perplexity}")